In [1]:
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly as plty
from plotly.offline import iplot
import numpy as np
plty.offline.init_notebook_mode(connected=True)


In [5]:
area = 5 #area of the circuit loop 

omega = 1 #angular velocity 

t = 0 #current time

totalt = 10 

dt = 0.01

A = np.array([0,0,area]) #area vector , rotates

B = np.array([2,0,0])  #Magnetic field vector  , remains constant 

j = np.array([0, 1, 0]) #unit vector vector for the y axis 

iterations = int(totalt/dt)

y = []
x = []



#This is the code that makes the plot for the circuit 
side = np.sqrt(area)/2 
#THIS CREATES THE CIRCUIT PLOT 
circuit = []

halfside = side*np.linspace(-1, 1, 20)

oneside = side*np.ones(halfside.shape)

x1 = np.append(halfside, oneside, axis = 0)
x1 = np.append(x1, -halfside,  axis = 0)
x1 = np.append(x1, -oneside, axis = 0)

y1 = np.append(oneside, halfside, axis = 0)
y1 = np.append(y1, -oneside,  axis = 0)
y1 = np.append(y1, halfside, axis = 0)

z = np.zeros(x1.shape)

circuit.append(x1)
circuit.append(y1)
circuit.append(z)



def circuitupdate(angle, circ):
    rotmat = np.matrix([[np.cos(angle), 0, -np.sin(angle)],[0, 1, 0],[np.sin(angle), 0, np.cos(angle)]])
    
    #circ[0] = circ[0].tolist()
    #circ[1] = circ[1].tolist()
    #circ[2] = circ[2].tolist()
    
    inc = len(circ[0])
    
    for i in range(0, inc):
        vec = np.array([circ[0][i], circ[1][i], circ[2][i]])
        newvec = rotmat.dot(vec)
        newvec = newvec.tolist()
        for k in range(0, 3):
            
            circ[k][i] = newvec[0][k]
            
            
    #circ[0] = np.array(circ[0])
    #circ[1] = np.array(circ[1])
    #circ[2] = np.array(circ[2])
    
    return(circ)

circuits = []



for i in range(0, iterations):
    t+=dt
    
    angle = omega*t
    
    rotmatrix = np.matrix([[np.cos(angle), 0, -np.sin(angle)],[0, 1, 0],[np.sin(angle), 0, np.cos(angle)]])
    
    newA = rotmatrix.dot(A)
    
    phi = np.cross(newA, B)
    
    
    if phi.dot(j)[0] < 0:
        phi = -np.linalg.norm(phi)
    else:
        phi = np.linalg.norm(phi)
    
    y.append(phi)
    x.append(t)
    
    circ = circuit

    inc = len(circ[0])

    for i in range(0, inc):
        vec = np.array([circ[0][i], circ[1][i], circ[2][i]])
        newvec = rotmatrix.dot(vec)
        newvec = newvec.tolist()
        for k in range(0, 3):

            circ[k][i] = newvec[0][k]


        #circ[0] = np.array(circ[0])
        #circ[1] = np.array(circ[1])
        #circ[2] = np.array(circ[2])

    circuits.append(circ)
    
    
    #newcircuit = circuitupdate(angle, circuit)
    
    #circuits.append(circuitupdate(angle, circuit))
    
d = np.diff(y)/np.diff(x)

emf = -d
    
    
#graphs

points = go.Scatter(x=x, y=y,
                    mode='lines',
                    marker=dict(size=2),
                    name='points')

contourlayout = dict(    
    width=800,height=800,    
    showlegend=True,           
    scene = dict(        
        xaxis = dict(range=[-5, 5], autorange=False, zeroline=False),        
        yaxis = dict(range=[-5, 5], autorange=False, zeroline=False),
    ),    
    plot_bgcolor= 'rgb(255, 255, 255)')

fig = go.Figure(data=[points], layout = contourlayout)

#iplot(fig)

points = go.Scatter(x=x, y=emf,
                    mode='lines',
                    marker=dict(size=2),
                    name='points')
fig = go.Figure(data=[points], layout = contourlayout)
#iplot(fig)

frame = 50

circ = circuitupdate(np.pi/2, circuit)

trace = go.Scatter3d(
    x = circ[0], 
    y = circ[1], 
    z = circ[2],
    mode='lines',
    line=dict(
        width=2
        )
)


data = [trace]

layout = go.Layout(
                    scene = dict(
                    xaxis = dict(
                        nticks=4, range = [-2,2],),
                    yaxis = dict(
                        nticks=4, range = [-2,2],),
                    zaxis = dict(
                        nticks=4, range = [-2,2],),),
                    width=700,
                    margin=dict(
                    r=20, l=10,
                    b=10, t=10)
                  )

fig = go.Figure(data=data, layout=layout)

iplot(fig)